# **Deep Hedging**
# Buchkov Viacheslav

In [ ]:
!git clone https://github.com/v-buchkov/deep-hedging.git

In [ ]:
import sys

sys.path.insert(0, "/content/deep-hedging")

In [ ]:
from pathlib import Path

from deep_hedging import ExperimentConfig, EuropeanCall, seed_everything
from deep_hedging.dl import SpotDataset, Trainer, Assessor
from deep_hedging.dl.models import LSTMHedger
from deep_hedging.dl.baselines import BaselineEuropeanCall

RANDOM_SEED = 12

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

PATH = Path("/content/gdrive/MyDrive/")

In [ ]:
config = ExperimentConfig(
    RANDOM_SEED=RANDOM_SEED, DATA_ROOT=PATH, OUTPUT_ROOT=Path("."), DATA_FILENAME="2024"
)
seed_everything(RANDOM_SEED)

config.DEVICE

In [ ]:
print(config)

## Dataset.

In [ ]:
spot_dataset = SpotDataset(
    n_days=config.N_DAYS, instrument_cls=EuropeanCall, config=config
)
spot_dataset.df.head()

## Run.

In [ ]:
N_EPOCHS = 10
config.REBAL_FREQ = "30 min"

trainer = Trainer(model_cls=LSTMHedger, instrument_cls=EuropeanCall, config=config)
trainer.run(N_EPOCHS)

## Baseline check.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from deep_hedging.dl import Assessor

assessor = Assessor(
    model=trainer.hedger,
    baseline=BaselineEuropeanCall(dt=trainer.dt).to(config.DEVICE),
    test_loader=trainer.test_loader,
)
assessor.run()

In [ ]:
trainer.save(config.OUTPUT_ROOT)